In [30]:
import pandas as pd
import graph_tool.all as gt

In [38]:
import os
from functools import reduce

In [31]:
path = '../../data/us/raw'

In [66]:
for session in ['113', '114', '115']:
    # make graph
    cosponsorships = pd.concat([pd.read_csv(os.path.join(path,session,chamber,'cosponsorships.csv'),header=None)
                                        for chamber in os.listdir(os.path.join(path, session))])
    g = gt.Graph(directed=False)
    g.vp['id'] = g.add_edge_list(cosponsorships.values, hashed=True)
    gt.remove_parallel_edges(g)
    gt.remove_self_loops(g)
    
    # add property maps
    party_df = parties[session]
    g.vp['party'] = g.new_vp('string', party_df[party_df['bioguide_id'].isin(list(g.vp['id']))]['party'].values)
    
    # save
    g.save(f'../../data/us/graphs/us.{session}.graphml')